In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [42]:
tf.__version__

'2.3.1'

In [43]:
data = pd.read_csv('../data/Churn_Modelling.csv')

In [44]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [45]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [46]:
data['Surname'].value_counts(1)

Smith        0.0032
Scott        0.0029
Martin       0.0029
Walker       0.0028
Brown        0.0026
              ...  
Perreault    0.0001
Outhwaite    0.0001
Foxall       0.0001
Tyndall      0.0001
Efremova     0.0001
Name: Surname, Length: 2932, dtype: float64

In [47]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [48]:
continuas=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','EstimatedSalary',]

In [49]:
criterio_iqr={}
for variable in continuas:
    aux=data[[variable]].copy()
    registros_anteriores=aux.shape[0]
    q1=aux[variable].quantile(.25)
    q3=aux[variable].quantile(.75)
    iqr=q3-q1
    aux=aux[~((aux[variable]<q1-1.5*iqr)|(aux[variable]>q3+1.5*iqr))]
    registros_despues_outliers=aux.shape[0]
    criterio_iqr[variable]={'registros_previos':registros_anteriores,'registros_despues_limpieza':registros_despues_outliers}

In [50]:
pd.DataFrame(criterio_iqr).T

,registros_previos,registros_despues_limpieza
CreditScore,10000,9985
Age,10000,9641
Tenure,10000,10000
Balance,10000,10000
NumOfProducts,10000,9940
HasCrCard,10000,10000
EstimatedSalary,10000,10000


In [51]:
criterio_media={}
for variable in continuas:
    n=2
    aux=data[[variable]].copy()
    registros_anteriores=aux.shape[0]
    media=aux[variable].mean()
    desv=aux[variable].std()
    aux=aux[(aux[variable]<= media+2*desv)&(aux[variable]>=(media-2*desv))]
    registros_despues_outliers=aux.shape[0]
    criterio_media[variable]={'registros_previos':registros_anteriores,'registros_despues_limpieza':registros_despues_outliers}
    

In [52]:
pd.DataFrame(criterio_media).T

,registros_previos,registros_despues_limpieza
CreditScore,10000,9500
Age,10000,9474
Tenure,10000,10000
Balance,10000,9970
NumOfProducts,10000,9674
HasCrCard,10000,10000
EstimatedSalary,10000,10000


In [53]:
data_clean=data.copy()
for variable in continuas:
    q1=data_clean[variable].quantile(.25)
    q3=data_clean[variable].quantile(.75)
    iqr=q3-q1
    data_clean=data_clean[~((data_clean[variable]<q1-1.5*iqr)|(data_clean[variable]>q3+1.5*iqr))]

In [54]:
data_clean.shape

(9568, 14)

In [55]:
9568/data.shape[0]

0.9568

In [56]:
data_clean.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [57]:
data_clean['Geography'].value_counts(1)

France     0.501463
Germany    0.250627
Spain      0.247910
Name: Geography, dtype: float64

In [58]:
data_clean['Gender']=data_clean['Gender'].map(lambda x : 1 if x=='Female' else 0)

In [59]:
data_clean=pd.get_dummies(data=data_clean,columns=['Geography'])

In [60]:
data_clean.head()

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [61]:
data_clean.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Geography_France', 'Geography_Germany',
       'Geography_Spain'],
      dtype='object')

In [62]:
variables=['CreditScore', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary','Geography_France', 'Geography_Germany']

In [63]:
X=data_clean[variables]
y=data_clean[['Exited']]

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [65]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,Input

In [67]:
ann = Sequential()

In [68]:
X_train.shape[1]

11

In [69]:
ann.add(Input(shape=X_train.shape[1]))

In [70]:
ann.add(Dense(6, activation='relu'))

In [71]:
ann.add(Dense(units=9, activation='relu'))

In [72]:
ann.add(Dense(units=1, activation='sigmoid'))

In [73]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [74]:
data_clean.shape

(9568, 16)

In [75]:
data.shape


(10000, 14)

In [76]:
X_train.shape[0]/32

239.1875

In [77]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
240/240 [==============================] - 0s 984us/step - loss: 0.5198 - accuracy: 0.7822
Epoch 2/100
240/240 [==============================] - 0s 1ms/step - loss: 0.4373 - accuracy: 0.8158
Epoch 3/100
240/240 [==============================] - 0s 1ms/step - loss: 0.4170 - accuracy: 0.8281
Epoch 4/100
240/240 [==============================] - 0s 1ms/step - loss: 0.4082 - accuracy: 0.8309
Epoch 5/100
240/240 [==============================] - 0s 1ms/step - loss: 0.4005 - accuracy: 0.8309
Epoch 6/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3918 - accuracy: 0.8346
Epoch 7/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3831 - accuracy: 0.8380
Epoch 8/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3743 - accuracy: 0.8435
Epoch 9/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3673 - accuracy: 0.8483
Epoch 10/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3626 - accu

240/240 [==============================] - 0s 2ms/step - loss: 0.3415 - accuracy: 0.8571
Epoch 82/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3412 - accuracy: 0.8579
Epoch 83/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3411 - accuracy: 0.8577
Epoch 84/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8596
Epoch 85/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3413 - accuracy: 0.8573
Epoch 86/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8588
Epoch 87/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3413 - accuracy: 0.8581
Epoch 88/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3407 - accuracy: 0.8590
Epoch 89/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3411 - accuracy: 0.8593
Epoch 90/100
240/240 [==============================] - 0s 1ms/step - loss: 0.3409 - accuracy: 

In [78]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [79]:
def metricas(ypred,yv): #Mide efectividad de un Modelo Predictivo
    print( " Acc Validate: %.3f" %accuracy_score(y_pred=ypred,y_true=yv))
    print( " Matrix Conf Validate: ", "\n",confusion_matrix(y_pred=ypred,y_true=yv))

In [80]:
ann.predict(X_test)

array([[0.97600174],
       [0.19326895],
       [0.03058875],
       ...,
       [0.17596844],
       [0.00685996],
       [0.4938668 ]], dtype=float32)

In [81]:
y_pred =pd.DataFrame( ann.predict(X_test),columns=['proba'])


In [82]:
y_pred

,proba
0,0.976002
1,0.193269
2,0.030589
3,0.274372
4,0.005997
...,...
1909,0.059068
1910,0.109611
1911,0.175968
1912,0.006860


In [83]:

y_pred['clase']=y_pred['proba'].map(lambda x : 1 if x>.5 else 0)

In [84]:
y_pred

,proba,clase
0,0.976002,1
1,0.193269,0
2,0.030589,0
3,0.274372,0
4,0.005997,0
...,...,...
1909,0.059068,0
1910,0.109611,0
1911,0.175968,0
1912,0.006860,0


In [85]:
metricas(y_pred['clase'],y_test)

 Acc Validate: 0.875
 Matrix Conf Validate:  
 [[1512   59]
 [ 181  162]]


Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000


In [86]:
X.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany'],
      dtype='object')

In [39]:
nueva_observacion=[600,0,40,3,60000,2,1,1,50000,1,0]

In [87]:
ann.predict(sc.transform([nueva_observacion]))

array([[0.02190644]], dtype=float32)